In [1]:
import math
import os
import random
import shutil
import PIL
import pandas as pd
import numpy as np
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import string
import cv2
import tensorflow as tf
from PIL.Image import Image
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
#from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers import Concatenate
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.preprocessing import image
#from keras.utils import to_categorical
from keras import initializers
#from keras.engine.topology import Layer
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
#import SciPy
import keras
from tensorflow.keras import metrics
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.metrics import confusion_matrix

2022-12-20 21:25:14.659014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import shutil
def writer_to_folder(path,newpath):
    dic={}
    for filename in  os.listdir(path):
        writer=filename.split('_')[0]
        if writer in dic:
            dic[writer].append(filename)
        else:
            dic[writer]=[]
            dic[writer].append(filename)

    
    for keyed in dic.items():
        os.makedirs(newpath+'/'+keyed[0])
        for i in keyed[1]:
            try:
                shutil.copy(path+'/'+i, newpath+'/'+keyed[0]+'/')
                print("File copied successfully.")
            except shutil.SameFileError:
                 print("Source and destination represents the same file.")
 
        
    # data = pd.DataFrame(columns=["image", "writer"])

dicc=writer_to_folder('./Dataset/Train','./DatasetTrainWriters')
print(dicc)


FileExistsError: [Errno 17] File exists: './DatasetTrainWriters/AHTD3A1058'

In [3]:
def loadImages(path = "./Dataset"):
    """
    returns a dataframe that saves image name and writer in each row
    """
    data = pd.DataFrame(columns=["image", "writer"])

    for writer in os.listdir(path):
        writer_path = os.path.join(path,writer)

        for image in os.listdir(os.path.join(path,writer)):
            data.loc[len(data)] = [image, writer]

        data.to_csv('Writerset.csv', index=False)

    return data



In [4]:
loadImages("./DatasetTrainWriters")

,image,writer
0,AHTD3A0441_Para3.tif,AHTD3A0441
1,AHTD3A0441_Para2.tif,AHTD3A0441
2,AHTD3A0774_Para3.tif,AHTD3A0774
3,AHTD3A0774_Para2.tif,AHTD3A0774
4,AHTD3A0934_Para3.tif,AHTD3A0934
...,...,...
1395,AHTD3A0500_Para2.tif,AHTD3A0500
1396,AHTD3A0710_Para3.tif,AHTD3A0710
1397,AHTD3A0710_Para2.tif,AHTD3A0710
1398,AHTD3A0655_Para3.tif,AHTD3A0655


In [33]:

def split_into_patches(path, img, img_name, num_of_patches):
    # set borders to the image
    ymax, xmax = img.shape[0], img.shape[1]
    # for the patch name
    counter = 0
    # set the height and width of the patch size
    h, w = patch_size, patch_size
    # keep track of the patches that created
    x1, x2 = 0, w
    y1, y2 = 0, h

    while (counter < num_of_patches):
        if (x2 > xmax):
            x1 = 0
            x2 = w
            y1 += 240
            y2 += 240
        if (y2 > ymax):
            break
        temp = img[y1:y2, x1:x2]
        if ((x2 - x1) == patch_size and (y2 - y1) == patch_size and checkPatch(temp)):
            cv2.imwrite(path + '/' + img_name + '_' + str(counter) + '.tif', temp)
            counter += 1
        x1 += 400
        x2 += 400
    if counter < num_of_patches:
        split_into_patches_random(path, img, img_name, num_of_patches, counter)


def split_into_patches_random(path, img, img_name, num_of_patches, counter):
    # set borders to the image
    ymax, xmax = img.shape[0], img.shape[1]
    # set the height and width of the patch size
    h, w = patch_size, patch_size
    # keep track of the number of patches created
    # number of rows in the document
    numOfRows = (img.shape[0] // row_size) + 1
    print(patch_size,numOfRows)
    print(xmax,ymax)
    print(counter,num_of_patches)

    while (counter < num_of_patches):
        # print(counter)
        # radom start x point of patch
        x_startPatch = random.randrange(0, xmax - patch_size)

        # radom start y point of patch
        if (numOfRows - 4 > 0):
            randomRowNumber = (random.randrange(0, numOfRows - 3))
        else:
            randomRowNumber = 0

        y_startPatch = randomRowNumber * row_size
        # end x point
        x_endPatch = x_startPatch + patch_size
        # end y point
        y_endPatch = y_startPatch + patch_size

        patch = img[y_startPatch:y_endPatch, x_startPatch:x_endPatch]

        if (checkPatch(patch)):
            counter += 1
            cv2.imwrite(path + '/' + img_name + '_' + str(counter) + '.tif', patch)


def checkPatch(img):
    h,w= img.shape[0],img.shape[1]
    threshold = 400000

    orig = img.copy()
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,img = cv2.threshold(img,240,255,cv2.THRESH_BINARY_INV)

    #Split to four quarters
    upLeft = img[0:h//2,0:w//2]
    upRight = img[0:h//2,w//2:]
    downLeft = img[h//2:,0:w//2]
    downRight = img[h//2:,w//2:]

    #claculate the sum of each quarter
    sumUpLeft = np.sum(upLeft)
    sumUpRight = np.sum(upRight)
    sumDownLeft = np.sum(downLeft)
    sumDownRight = np.sum(downRight)

    #check if bigger than the threshold
    boolUpLeft = threshold < sumUpLeft
    boolUpRight = threshold < sumUpRight
    boolDownLeft = threshold < sumDownLeft
    boolDownRight = threshold < sumDownRight

    #check if at least 3 quarters are bigger than thershold
    isValidPatch = (int(boolUpRight) + int(boolUpLeft) + int(boolDownLeft) + int(boolDownRight)) > 2

    return isValidPatch


def find_squares2(img_bgr):
    location = []
    img = img_bgr
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    marker_count, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0

    segmented = cv2.watershed(img, markers)

    # END of original watershed example

    output = np.zeros_like(img)
    output2 = img.copy()
    xmin, xmax, ymax = 1000000, 0, 0

    # Iterate over all non-background labels
    for i in range(2, marker_count + 1):
        mask = np.where(segmented == i, np.uint8(255), np.uint8(0))
        x, y, w, h = cv2.boundingRect(mask)
        area = cv2.countNonZero(mask[y:y + h, x:x + w])
        location.append([x, y])

        if x < xmin:
            xmin = x
        if x > xmax:
            xmax = x
        if y > ymax:
            ymax = y

        # Visualize
        color = random.randint(0, 255 + 1)
        output[mask != 0] = color
        cv2.rectangle(output2, (x, y), (x + w, y + h), color, 1)
        cv2.putText(output2, '%d' % i, (x + w // 4, y + h // 2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1,
                    cv2.LINE_AA)


    # angle = calc_angle([location[0][0], location[1][0]], [location[0][1], location[1][1]])
    angle=0
    return angle, xmin, xmax, ymax


def calc_angle(points_x, points_y):
    # Calculating the angle of the image rotation.
    # according to the squares parameters.
    if points_x[0] < points_x[1]:
        y = points_y[0]
        points_y[0] = points_y[1]
        points_y[1] = y

    a = abs(points_y[1] - points_y[0])
    b = abs(points_x[1] - points_x[0])
    c = math.sqrt(a * a + b * b)
    angle = math.acos(a / c)
    angle = 90 - math.degrees(angle)

    if points_y[1] > points_y[0]:
        return -angle
    else:
        return angle

def rotate_image(img,angle):
    # Rotate a given image.
    (h, w) = img.shape[:2]
    center = (w / 2, h / 2)

    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    #print("rotate {0}° right".format(angle))
    return rotated

def boundText(im):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    _,thresh=cv2.threshold(gray,210,255,cv2.THRESH_BINARY)
    thresh = 255-thresh

    image, contours  = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #print(contours)
    newContours = []
    x1min = 10
    y1min = 10
    x2max = 0
    y2max = 0
    for cnt in image:
        if cv2.contourArea(cnt) > 150:
            [x, y, w, h] = cv2.boundingRect(cnt)
            if x<x1min:
                x1min = x
            if y<y1min:
                y1min = y
            if x+w>x2max:
                x2max = x+w
            if y+h>y2max:
                y2max = y+h
    return im[y1min:y2max,x1min:x2max]

In [6]:
# def convert_document_to_patches(path = "./All_images"):
#     document_name_list = os.listdir(path)

#     path_dir = './patches'
#     if (os.path.isdir(path_dir)) != True:
#         os.mkdir(path_dir)
#     for doc_name in document_name_list:

#         print(doc_name)
#         path_to_save_patches = path_dir + '/' + doc_name[:len(doc_name)-4]
#         if (os.path.isdir(path_to_save_patches)) != True:
#             os.mkdir(path_to_save_patches)
#         try:
#             img = extractTextbox(doc_name)
#             split_into_patches(path_to_save_patches,img,doc_name[:len(doc_name)-4],5)
#         except AttributeError:
#             print('cannot find text box')

def extractTextbox(document_name):
    print(document_name)
    img = PIL.Image.open(document_name)
    img = img.resize((w, h))
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_BAYER_BG2BGR)
    angle, xmin, xmax, ymax = find_squares2(img)
    print(angle,xmin,xmax,ymax)

    # img = rotate_image(img, angle)
    # img = remove_yellow(cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
    img = np.array(img)
    # img = img[20:ymax - 20, xmin:xmax + 100]
    img = boundText(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    return img


In [7]:
h,w= 1700,1700
patch_size = 400
row_size = 120


In [44]:
def convert_document_to_patches(path = "./Dataset/Train"):
    document_name_list = os.listdir(path)

    path_dir = './patches'
    if (os.path.isdir(path_dir)) != True:
        os.mkdir(path_dir)
    for doc_name in document_name_list:

        print(doc_name)
        path_to_save_patches = path_dir + '/' + doc_name[:len(doc_name)-10]
        print(path_to_save_patches)
        if (os.path.isdir(path_to_save_patches)) != True:
            os.mkdir(path_to_save_patches)
        try:
            img = extractTextbox(path+'/'+doc_name)
            split_into_patches(path_to_save_patches,img,doc_name[:len(doc_name)-4],5)
        except AttributeError:
            print('cannot find text box')

convert_document_to_patches()

AHTD3A1058_Para2.tif
./patches/AHTD3A1058
./Dataset/Train/AHTD3A1058_Para2.tif
0 542 1622 400
400 4
1690 432
3 5
AHTD3A0707_Para2.tif
./patches/AHTD3A0707
./Dataset/Train/AHTD3A0707_Para2.tif
0 1210 1277 467
AHTD3A0466_Para2.tif
./patches/AHTD3A0466
./Dataset/Train/AHTD3A0466_Para2.tif
0 799 799 681
AHTD3A0341_Para2.tif
./patches/AHTD3A0341
./Dataset/Train/AHTD3A0341_Para2.tif
0 0 1309 792
AHTD3A0836_Para3.tif
./patches/AHTD3A0836
./Dataset/Train/AHTD3A0836_Para3.tif
0 405 1506 1197
AHTD3A0995_Para3.tif
./patches/AHTD3A0995
./Dataset/Train/AHTD3A0995_Para3.tif
0 12 1620 1590
AHTD3A1076_Para2.tif
./patches/AHTD3A1076
./Dataset/Train/AHTD3A1076_Para2.tif
0 543 1564 1590
AHTD3A0935_Para3.tif
./patches/AHTD3A0935
./Dataset/Train/AHTD3A0935_Para3.tif
0 1 1654 1519
AHTD3A0580_Para3.tif
./patches/AHTD3A0580
./Dataset/Train/AHTD3A0580_Para3.tif
0 208 1665 1581
AHTD3A0649_Para3.tif
./patches/AHTD3A0649
./Dataset/Train/AHTD3A0649_Para3.tif
0 19 1681 1592
AHTD3A0160_Para2.tif
./patches/AHTD3A0160

In [24]:
loc='./Dataset/Train/AHTD3A0881_Para2.tif'
# img = PIL.Image.open(loc)
# img = np.array(img)
# img= cv2.cvtColor(img, cv2.COLOR_BAYER_BG2BGR )
img=extractTextbox(loc)
# cv2.imshow('Image', img)
#split_into_patches()

./Dataset/Train/AHTD3A0881_Para2.tif
0 360 1503 1544


In [25]:
cv2.imwrite('./Test/test.tif',img)


True

In [34]:
split_into_patches('./Test/patches', img, "img_name", 4)

400 14
1700 1660
2 4


In [ ]:
def convert_document_to_patches(path = "./Dataset/Train"):
    document_name_list = os.listdir(path)
    print(document_name_list)
convert_document_to_patches()

In [5]:
def patchesCsv():
    """
    returns a dataframe that saves patch name and writer in each row
    """
    writerdf = pd.read_csv('Writerset.csv')
    patches = []
    for imageFolder in os.listdir("./patches"):
        print("image: ", imageFolder)
        image = imageFolder
        # row = writerdf[writerdf["image"] == image]
        writer = int(imageFolder[len(imageFolder)-4:])

        for patch in os.listdir(os.path.join("./patches",imageFolder)):
            patches.append((patch,writer))

    patchesdf = pd.DataFrame(patches, columns=["image", "writer"])
    patchesdf.to_csv('patches.csv', index=False)

    return patchesdf


In [43]:
patchesCsv()

image:  AHTD3A0441
image:  AHTD3A0774
image:  AHTD3A0934
image:  AHTD3A0594
image:  AHTD3A0998
image:  AHTD3A0886
image:  AHTD3A1121
image:  AHTD3A0131
image:  AHTD3A0933
image:  AHTD3A1025
image:  AHTD3A0499
image:  AHTD3A0954
image:  AHTD3A0603
image:  AHTD3A1142
image:  AHTD3A0742
image:  AHTD3A0445
image:  AHTD3A0425
image:  AHTD3A0455
image:  AHTD3A0586
image:  AHTD3A0607
image:  AHTD3A0628
image:  AHTD3A1011
image:  AHTD3A0582
image:  AHTD3A0467
image:  AHTD3A0312
image:  AHTD3A0437
image:  AHTD3A0979
image:  AHTD3A0739
image:  AHTD3A0631
image:  AHTD3A0336
image:  AHTD3A0377
image:  AHTD3A0462
image:  AHTD3A0663
image:  AHTD3A1144
image:  AHTD3A0775
image:  AHTD3A0636
image:  AHTD3A0223
image:  AHTD3A0453
image:  AHTD3A0540
image:  AHTD3A0681
image:  AHTD3A0797
image:  AHTD3A0471
image:  AHTD3A0550
image:  AHTD3A0578
image:  AHTD3A0240
image:  AHTD3A0893
image:  AHTD3A1026
image:  AHTD3A0308
image:  AHTD3A0295
image:  AHTD3A0673
image:  AHTD3A0452
image:  AHTD3A1159
image:  AHTD

,image,writer
0,AHTD3A0441_Para2_3.tif,441
1,AHTD3A0441_Para2_4.tif,441
2,AHTD3A0441_Para2_0.tif,441
3,AHTD3A0441_Para2_1.tif,441
4,AHTD3A0441_Para3_2.tif,441
...,...,...
7863,AHTD3A0655_Para2_5.tif,655
7864,AHTD3A0655_Para2_0.tif,655
7865,AHTD3A0655_Para3_1.tif,655
7866,AHTD3A0655_Para3_3.tif,655


In [2]:
def applyAugmentation(img):
    img = img.reshape(1, img.shape[0], img.shape[1], 1)
    datagen = ImageDataGenerator(rotation_range=15, fill_mode='nearest')
    aug_iter = datagen.flow(img, batch_size=1)

    image1 = next(aug_iter)[0].astype('uint8')
    image2 = next(aug_iter)[0].astype('uint8')
    image3 = next(aug_iter)[0].astype('uint8')

    return image1, image2, image3





def generateAugImages(path = "./patches", dest = "./aug_patches"):

    patch_df = pd.read_csv("patches.csv")

    imageList = []
    writerList = []

    for folder in os.listdir(path): # folder name is image name(without patch num)
        print(folder)
        img_dir = os.path.join(path,folder)

        for patch in os.listdir(img_dir):
            print("patch: ", patch)
            image = cv2.imread(img_dir + "/" + patch, cv2.IMREAD_GRAYSCALE)
            img1, img2, img3 = applyAugmentation(image)
            dir_dest = os.path.join(dest,folder)

            if (os.path.isdir(dir_dest)) != True:
                os.mkdir(dir_dest)

            new_img1 = patch[:len(patch) - 4] + "_1.tif"
            new_img2 = patch[:len(patch) - 4] + "_2.tif"
            new_img3 = patch[:len(patch) - 4] + "_3.tif"

            cv2.imwrite(dir_dest + "/" + new_img1, img1)
            cv2.imwrite(dir_dest + "/" + new_img2, img2)
            cv2.imwrite(dir_dest + "/" + new_img3, img3)
            cv2.imwrite(dir_dest + "/" + patch, image)

            imageList.append(new_img1)
            imageList.append(new_img2)
            imageList.append(new_img3)
            imageList.append(patch)

            writer = patch_df.loc[patch_df['image'] == patch, 'writer'].iloc[0]
            for i in range(4):
                writerList.append(writer)

    aug_imgs = pd.DataFrame({"image": imageList, "writer": writerList})
    aug_imgs.to_csv('patches_aug.csv', index=False)


In [5]:
generateAugImages()

AHTD3A0441
patch:  AHTD3A0441_Para2_3.tif
patch:  AHTD3A0441_Para2_4.tif
patch:  AHTD3A0441_Para2_0.tif
patch:  AHTD3A0441_Para2_1.tif
patch:  AHTD3A0441_Para3_2.tif
patch:  AHTD3A0441_Para3_4.tif
patch:  AHTD3A0441_Para3_1.tif
patch:  AHTD3A0441_Para2_2.tif
patch:  AHTD3A0441_Para3_3.tif
patch:  AHTD3A0441_Para3_0.tif
AHTD3A0774
patch:  AHTD3A0774_Para3_1.tif
patch:  AHTD3A0774_Para2_3.tif
patch:  AHTD3A0774_Para3_2.tif
patch:  AHTD3A0774_Para2_1.tif
patch:  AHTD3A0774_Para3_0.tif
patch:  AHTD3A0774_Para2_2.tif
patch:  AHTD3A0774_Para3_4.tif
patch:  AHTD3A0774_Para2_4.tif
patch:  AHTD3A0774_Para2_0.tif
patch:  AHTD3A0774_Para3_3.tif
AHTD3A0934
patch:  AHTD3A0934_Para2_0.tif
patch:  AHTD3A0934_Para2_1.tif
patch:  AHTD3A0934_Para3_1.tif
patch:  AHTD3A0934_Para2_2.tif
patch:  AHTD3A0934_Para3_0.tif
patch:  AHTD3A0934_Para3_4.tif
patch:  AHTD3A0934_Para2_4.tif
patch:  AHTD3A0934_Para3_2.tif
patch:  AHTD3A0934_Para2_3.tif
patch:  AHTD3A0934_Para3_3.tif
AHTD3A0594
patch:  AHTD3A0594_Para3_3

In [5]:
def deepnn(x, train):
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 2048])
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([2048, 1024])
    b_fc1 = bias_variable([1024]) 
    h_fc1 = tf.nn.relu(tf.matmul(x_image, W_fc1) + b_fc1)
    
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 512])
    b_fc2 = bias_variable([512]) 
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

  with tf.name_scope('fc3'):
    W_fc3 = weight_variable([512, 128])
    b_fc3 = bias_variable([128]) 
    h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)    
    
  with tf.name_scope('fc4'):
    W_fc4 = weight_variable([128, 1])
    b_fc4 = bias_variable([1])
    y_conv = tf.sigmoid(tf.matmul(h_fc3, W_fc4) + b_fc4)
    return y_conv, h_fc3

def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.05)
  return tf.Variable(initial)

def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


In [6]:
def base_network(input_dim):
    inputs = Input(shape=input_dim)
    conv_1=Conv2D(64,(5,5),padding="same",activation='relu',name='conv_1')(inputs)
    conv_1=MaxPooling2D(pool_size=(2, 2))(conv_1)
    conv_2=Conv2D(128,(5,5),padding="same",activation='relu',name='conv_2')(conv_1)
    conv_2=MaxPooling2D(pool_size=(2, 2))(conv_2)
    conv_3=Conv2D(256,(3,3),padding="same",activation='relu',name='conv_3')(conv_2)
    conv_3=MaxPooling2D(pool_size=(2, 2))(conv_3)
    conv_4=Conv2D(512,(3,3),padding="same",activation='relu',name='conv_4')(conv_3)
    conv_5=Conv2D(512,(3,3),padding="same",activation='relu',name='conv_5')(conv_4)
    conv_5=MaxPooling2D(pool_size=(2, 2))(conv_5)

    dense_1=Flatten()(conv_5)
    dense_1=Dense(512,activation="relu")(dense_1)
    dense_1=Dropout(0.3)(dense_1)
    dense_2=Dense(512,activation="relu")(dense_1)
    dense_2=Dropout(0.5)(dense_2)
    return Model(inputs, dense_2)


In [7]:
input_shape=(150,150,1)
learning_rate=0.000001

base_network = base_network(input_shape)
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
processed_a = base_network(input_a)
processed_b = base_network(input_b)

fc6=concatenate([processed_a, processed_b])

fc7 = Dense(4096, activation = 'relu')(fc6)
fc7 = Dropout(0.7)(fc7)
fc8 = Dense(4096, activation = 'relu')(fc7)
fc8 = Dropout(0.8)(fc8)

fc9=Dense(1, activation='sigmoid')(fc8)
model = Model([input_a, input_b], fc9)

2022-12-20 17:51:36.726738: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 17:51:37.578984: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 17:51:37.579071: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 17:51:37.580122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other oper

In [8]:
def splitData(path = "Writerset.csv"):
    """
    splits data to train and test
    returns test writers for page level functions
    """
    data = pd.read_csv(path)

    writers = set(data["writer"])
    writers = list(writers)
    random.shuffle(writers)
    writers_len = len(list(writers))
    train = writers[:round(writers_len * 0.80)]
    test = writers[round(writers_len * 0.80):]

    train_df = data.loc[data['writer'].isin(train)]
    test_df = data.loc[data['writer'].isin(test)]

    return train_df, test_df, test


In [9]:
def loadImageDict(df, path = "./aug_patches", khatt = 0):
    """
    :param df: df which we will load images from (columns: image, writer)
    :param path: path to load images from
    :return: a dict with image names as keys and loaded images as values
    """
    dict = {}
    for image in df['image'].values:
        if khatt == 0: # regular data
            index = image.find("_", image.find("_")+1) # find index of second _ in order to find folder name(= original image name)
            folder = image[:index]  # original image name
        if khatt == 1:
            if (len(image)==24):
             folder = image[:len(image) - 14] #original image name
            else:
             folder = image[:len(image) - 12] #original image name

            print(folder)
        folder_path = os.path.join(path,folder)

        # load image
        loaded = cv2.resize(cv2.imread(folder_path + "/" + image, cv2.IMREAD_GRAYSCALE), (150, 150))
        thresh, loaded = cv2.threshold(loaded, 180, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        loaded = np.array(loaded, dtype=np.float32)
        #loaded/=255

        # add to dictionary
        dict[image] = loaded

    return dict

In [10]:
def newcreatePairs(df, dict, pairs_num):
    """
    :param df: df (with colums image, writer) to create pairs from
    :param dict: a dict with image names as keys and loaded images as values (from loadImageDict)
    :param pairs_num: pairs_num of pairs for same writer and pairs_num for different writer
    :return: 3 numpy arrays of left, right and labels
    """
    left = []
    right = []
    labels = []

    for index, row in df.iterrows():
        image = row["image"]
        writer = row["writer"]

        writer_images = df.loc[df['writer'] == writer]
        writer_images = writer_images["image"].values
        diff_writer_images = df.loc[df['writer'] != writer]
        diff_writer_images = diff_writer_images["image"].values

        # add pairs for same writer
        filtered_images = np.delete(writer_images, np.where(writer_images == image))
        try:
            right_patches = np.random.choice(filtered_images, size=pairs_num, replace=False)
        except:
            right_patches = np.random.choice(filtered_images, size=len(filtered_images), replace=False)
        for p in right_patches:
            left.append(dict[image])
            right.append(dict[p])
            labels.append(1)

        # add pairs for different writer
        try:
            right_patches = np.random.choice(diff_writer_images, size=pairs_num, replace=False)
        except:
            right_patches = np.random.choice(diff_writer_images, size=len(diff_writer_images), replace=False)
        for p in right_patches:
            left.append(dict[image])
            right.append(dict[p])
            labels.append(0)

        # shuffle
        # temp = list(zip(left, right, labels))
        # random.shuffle(temp)
        # left, right, labels = zip(*temp)
        # left, right, labels = list(left), list(right), list(labels)

    return np.array(left), np.array(right), np.array(labels).astype("float32")


In [11]:
 #training
adam=Adam(learning_rate=learning_rate)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


    #adam = Adam(learning_rate=learning_rate)
    #model = siamese_model((150, 150, 1))

train_df, test_df, _ = splitData("patches_aug.csv")
train_df = train_df.sample(frac = 1)
test_df = test_df.sample(frac = 1)

train_writers = train_df['writer'].unique()
test_writers = test_df['writer'].unique()

dic = loadImageDict(train_df, "./aug_patches", khatt = 1)
test_dic = loadImageDict(test_df, "./aug_patches", khatt = 1)
print("finished building dictionaries")

img_list = list(dic.values())
print(img_list[0].shape)


left, right, labels = newcreatePairs(train_df, dic, 1)
test_left, test_right, test_labels = newcreatePairs(test_df, test_dic, 1)

print("finished create pairs")

2022-12-20 17:51:57.538056: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-12-20 17:51:57.538874: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


AHTD3A0813
AHTD3A0544
AHTD3A0101
AHTD3A0655
AHTD3A0852
AHTD3A0957
AHTD3A0586
AHTD3A0557
AHTD3A0377
AHTD3A0875
AHTD3A0894
AHTD3A0977
AHTD3A0593
AHTD3A1101
AHTD3A1160
AHTD3A0282
AHTD3A0474
AHTD3A0898
AHTD3A0475
AHTD3A0641
AHTD3A0887
AHTD3A0553
AHTD3A1132
AHTD3A1061
AHTD3A0826
AHTD3A0575
AHTD3A0963
AHTD3A0383
AHTD3A0323
AHTD3A0223
AHTD3A0933
AHTD3A0155
AHTD3A0462
AHTD3A0119
AHTD3A0327
AHTD3A1008
AHTD3A0566
AHTD3A0150
AHTD3A0284
AHTD3A0408
AHTD3A0638
AHTD3A0525
AHTD3A0567
AHTD3A0881
AHTD3A0663
AHTD3A0101
AHTD3A0149
AHTD3A0602
AHTD3A0805
AHTD3A0798
AHTD3A0854
AHTD3A0698
AHTD3A0811
AHTD3A0215
AHTD3A1010
AHTD3A1158
AHTD3A0398
AHTD3A0308
AHTD3A0810
AHTD3A0642
AHTD3A0897
AHTD3A0625
AHTD3A0592
AHTD3A0406
AHTD3A0918
AHTD3A0716
AHTD3A0825
AHTD3A0797
AHTD3A0562
AHTD3A0948
AHTD3A0789
AHTD3A1132
AHTD3A0723
AHTD3A0456
AHTD3A0897
AHTD3A0449
AHTD3A0884
AHTD3A0924
AHTD3A1003
AHTD3A0539
AHTD3A0292
AHTD3A0361
AHTD3A0410
AHTD3A0670
AHTD3A0591
AHTD3A0716
AHTD3A0563
AHTD3A1077
AHTD3A0720
AHTD3A0697
AHTD3A0955

In [12]:
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=tf.keras.metrics.BinaryAccuracy( name="binary_accuracy", dtype=None, threshold=0.497))

model.fit([left, right], labels, epochs=2)

loss , acc = model.evaluate([test_left, test_right], test_labels, verbose=2)
print("accuracy: ", acc)

pred = model.predict([test_left, test_right])

: 

: 

In [11]:
train_df['image'].values

array(['AHTD3A0568_Para3_2_3.tif', 'AHTD3A0333_Para3_0.tif',
       'AHTD3A0446_Para2_3_3.tif', ..., 'AHTD3A0237_Para2_0.tif',
       'AHTD3A0491_Para2_2_3.tif', 'AHTD3A0844_Para2_2_2.tif'],
      dtype=object)

In [7]:
import tensorflow.compat.v1 as tff
tff.compat.v1.disable_eager_execution()

In [8]:
FLAGS = None

def deepnn(x, train):
  with tff.name_scope('reshape'):
    x_image = tff.reshape(x, [-1, 2048])
  with tff.name_scope('fc1'):
    W_fc1 = weight_variable([2048, 1024])
    b_fc1 = bias_variable([1024]) 
    h_fc1 = tff.nn.relu(tff.matmul(x_image, W_fc1) + b_fc1)
    
  with tff.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 512])
    b_fc2 = bias_variable([512]) 
    h_fc2 = tff.nn.relu(tff.matmul(h_fc1, W_fc2) + b_fc2)

  with tff.name_scope('fc3'):
    W_fc3 = weight_variable([512, 128])
    b_fc3 = bias_variable([128]) 
    h_fc3 = tff.nn.relu(tff.matmul(h_fc2, W_fc3) + b_fc3)    
    
  with tff.name_scope('fc4'):
    W_fc4 = weight_variable([128, 1])
    b_fc4 = bias_variable([1])
    y_conv = tff.sigmoid(tff.matmul(h_fc3, W_fc4) + b_fc4)
    return y_conv, h_fc3

def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tff.truncated_normal(shape, stddev=0.05)
  return tff.Variable(initial)

def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tff.constant(0.1, shape=shape)
  return tff.Variable(initial)

def get_batch(name_pos, label_pos, name_neg, label_neg, batch_size_p, batch_size_n, now_batch_p, now_batch_n, total_batch_n):
    image_batch_pos = name_pos[now_batch_p*batch_size_p:(now_batch_p+1)*batch_size_p,:]
    label_batch_pos = label_pos[now_batch_p*batch_size_p:(now_batch_p+1)*batch_size_p]
    image_batch_neg = name_neg[now_batch_n*batch_size_n:(now_batch_n+1)*batch_size_n,:]
    label_batch_neg = label_neg[now_batch_n*batch_size_n:(now_batch_n+1)*batch_size_n]        
    batchdata=np.concatenate((image_batch_pos,image_batch_neg),axis=0)        
    batchlabel=np.concatenate((label_batch_pos,label_batch_neg),axis=0)
    return batchdata, batchlabel

def loss_with_top_rank(logits, labels, p):
    
  with tff.name_scope('toprank'):
      
      loss = tff.zeros([1,1])
            
      num = tff.constant(1/5)
            
      index1 = tff.where(tff.equal(labels,1))
            
      indices1 = index1[:,0]
            
      index2 = tff.where(tff.not_equal(labels,1))
            
      indices2 = index2[:,0] 
            
      cs=tff.constant(1.0)
        
      p1 = p
        
      p2 = cs/p1
            
      sum_n = tff.zeros([1,1])
            
      sum_p = tff.zeros([1,1])
            
      l = tff.zeros([1,1])
            
      feature = tff.zeros([1,1])
            
      norm_feature = tff.zeros([1,1])
            
      l_p = tff.zeros([1,1])
            
      for u in range(5):
            
          for v in range(40):
                  
              feature = tff.subtract(logits[indices1[u]],logits[indices2[v]])
                
              norm_feature =tff.log(1+tff.exp(-feature))
                
              l_p = tff.pow(norm_feature,p1)
                
              sum_n = sum_n+l_p
    
          l = tff.pow(sum_n,p2)
              
          sum_p = sum_p+l
              
      loss = tff.multiply(sum_p,num)         
          
      return loss

In [9]:
for fold in range(5):
        p_array=[2,4,8,16,32]
        num_p=p_array[fold]
        p_value=np.float32(num_p)
        p_value=np.reshape(p_value,[1])
        tff.reset_default_graph()
        model_name = 'hindi_model'+'_p_' + str(num_p)
        training = tff.placeholder(tff.bool)    
        x = tff.placeholder(tff.float32, [None, 2048])
        y_ = tff.placeholder(tff.int64, [None,1])
        y_conv,fc1 = deepnn(x, training)

        with tff.name_scope('loss'):
    
            toprankloss = loss_with_top_rank(logits=y_conv, labels=y_, p=p_value)
    
            toprankloss = tff.reshape(toprankloss,[])

            initial_loss= tff.constant(1e-14)

            toprankloss = tff.add(toprankloss,initial_loss)

            loss_summary = tff.summary.scalar('loss', toprankloss)      

        global_step = tff.Variable(0, trainable=False)
        initial_learning_rate = 0.0001
        learning_rate = tff.train.exponential_decay(initial_learning_rate,
                                           global_step,
                                           decay_steps=100,decay_rate=0.90)
        with tff.name_scope('adam_optimizer'):
            update_ops = tff.get_collection(tff.GraphKeys.UPDATE_OPS)
        with tff.control_dependencies(update_ops):
            train_step = tff.train.AdamOptimizer(learning_rate).minimize(toprankloss)
        add_global = global_step.assign_add(1)

        sess=tff.Session()
        train_writer = tff.summary.FileWriter('./summary' + str(num_p)+ '/'+ 'p_' + str(num_p)+'_fc_hindi_3', sess.graph)
        sess.run(tff.global_variables_initializer())
        saver = tff.train.Saver()                  
        step=0
        for epoch in range(10):
            file=joblib.load('/home/xiaotong/Desktop/Experiments/Codes/Signature/dataset/BHSig260/'+di.dataset+'/resized/')
            train=file['train_pairs']
            traindata_pos,traindata_neg=train['G-G'],train['G-F']
            traindata_pos=np.array(traindata_pos)
            traindata_neg=np.array(traindata_neg)
            train_g_len=len(traindata_pos)
            train_f_len=len(traindata_neg)
            trainlabel_p=np.ones([train_g_len])
            trainlabel_n=np.zeros([train_f_len])
            now_batch_p=0
            now_batch_n=0
            batch_size_p=5
            batch_size_n=40
            total_batch_n=np.int(train_f_len/batch_size_n)+1
            num_of_pos=np.int(train_g_len/batch_size_p)
            num_of_neg=np.int(train_f_len/batch_size_n)
            train_p=traindata_pos
            train_n=traindata_neg
            for i in range(num_of_pos):
                now_batch_p=i
                if i < num_of_neg:
                    x_batch_train,y_batch_train=get_batch(train_p,trainlabel_p,train_n,trainlabel_n,batch_size_p,batch_size_n,now_batch_p,now_batch_n,total_batch_n)
                if i >= num_of_neg:
                    if now_batch_n % num_of_neg == 0:
                        now_batch_n = 0
                        train_n = traindata_neg
                    x_batch_train,y_batch_train = get_batch(train_p,trainlabel_p,train_n,trainlabel_n,batch_size_p,batch_size_n,now_batch_p,now_batch_n,total_batch_n)
                x_batch_train=np.float32(x_batch_train)
                y_batch_train=np.reshape(y_batch_train,[45,1])
                g, rate = sess.run([add_global, learning_rate])
                loss,losssummary, _ = sess.run([toprankloss,loss_summary,train_step], feed_dict={x: x_batch_train, y_ : y_batch_train, training:True})
                train_writer.add_summary(losssummary, step)
                print("step: {}, train loss: {:g}".format(step, loss))
                now_batch_n=now_batch_n+1                
                step=step+1
            if (epoch % 1 == 0):
                print("Saving checkpoint")
                saver.save(sess, '/home/xiaotong/Desktop/Experiments/Codes/Reliable_methods/TopRankNN_related/models/p_'+str(num_p)+'/' + model_name + '_fc.ckpt')

2022-12-20 21:32:13.599261: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 21:32:14.569766: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 21:32:14.570096: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-20 21:32:14.573701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other oper

PermissionDeniedError: /home/xiaotong; Permission denied